In [1]:
#importing all our files 
#imports of unused tools not added, refer experiment.ipynb file for complete runthrough
import pytesseract
from PIL import Image #not using 
import cv2 
import numpy as np
import google.generativeai as genai  
from pymongo import MongoClient  
import json 
import io #not using
import sys #not using


c:\Users\aruna\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We will be making the functions for our chosen operations
1) OCR Extraction
2) Classification
3) Adding to Database
4) update: another function was needed to parse the text as json
5) ONE COMBINED FUNCTION FOR THE ENTIRE PROCESS THAT CALLS ALL THE ABOVE FUNCTIONS

Making a function that will take the image location as input and then output the text extracted from the image

In [2]:
#function for image extraction
def image_extract(image_location):
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #tesseract path on pc
    image=Image.open(image_location)
    extracted_businesscard=pytesseract.image_to_string(image)
    return extracted_businesscard
   # print(extracted_businesscard)
    

image_extract("business cards/microsoft_business_card.png")
#worked

'Jordan\n\nMitchell\n\nCEO\n\nVanArsdel, Ltd.\n\n5678 Main St. New York, NY 90210\n212-555-0199\nwww.vanarsdelltd.com\n\nQ@evdanavanarsdetta.com\n'

1) make 3 functions one for extraction and one for classifying and one for putting into mongodb then make a single function combining them both
2) making 1 single function entirely
- I will be going with option 1 since it will make it easier for us to edit files

Calling the classifying function

Updated the classfying function to return null values as "0" since MongoDB doesnt accept the "null"

In [3]:

#FUNCTION MADE IN FUNCTION PAGE

#testing with new promts
def classify_text(image_location):
    input_text = image_extract(image_location)  # testing 2 with business card -> kk optics

    # Configure API key
    genai.configure(api_key="api_key")  # private key, I won't be putting it on github

    # Define a prompt for classification
    prompt =  f"""
Classify the following text into the following categories depending on if you feel like that value is present or not:
1. Company Name
2. Person Name
3. Job Title
4. Email
5. Location
6. Phone Numbers
7. Website
and more based on what you feel the info is about.
and return it in a format for storing it in a MongoDb database
if there are null values give it 0 instead of null
Input:
    {input_text}

"""


    # Generate the response
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
   # print(response.text)
    return response.text





In [4]:
test_text=classify_text("business cards/bs_card_3.jpg") #will return our text

Function to parse the text from as JSON for upload directly into MongoDB 

In [5]:
#trying to remove the ''' json '''' in the generated text



def extract_json_content(raw_text):
    try:
        # Find the first occurrence of '{' and the last occurrence of '}'
        start_index = raw_text.find('{')
        end_index = raw_text.rfind('}')
        
        # Extract the JSON content
        json_content = raw_text[start_index:end_index + 1]
        
        # Parse to ensure it's valid JSON
        parsed_json = json.loads(json_content)
        
        # Return the parsed JSON
        return parsed_json
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None
    


In [6]:
#testing the json function
#working with  business cards/bs_card_3.jpg
#not working with  business cards/bs_card_2.png
#update: bs card 2 was not clear
#worked with other cards
test_text=classify_text("business cards/bs_card_2.png")
test_text=extract_json_content(test_text)

The text is given in a dict format but will be recogonized by python as "str" which will prevent it from being added to the DB, so we are parsing it using the above function to make it a dict so that our DB accepts it

In [7]:
type(test_text) #dict format to upload to MongoDB

dict

The commented code is a rough code which hadnt been made a fucntion

In [8]:
#commented to avoid repeated inputs

# #trying to put that into the database 


# # Connect to local MongoDB instance
# client = MongoClient("mongodb://localhost:27017/") #our local mongodb server connection key

# # Access a database
# db = client["business_cards"]  # our database name

# # Access a collection
# collection = db["card_details"]  # oru collection name

# # Insert a document
# data =test_text
# collection.insert_one(data)

# print("Document inserted successfully!")



Makiing a function to add a given value to the database

In [9]:
def add_db(test_text):
    # Connect to local MongoDB instance
    client = MongoClient("mongodb://localhost:27017/") #our local mongodb server connection key

        # Access a database
    db = client["business_cards"]  # our database name

    # Access a collection
    collection = db["card_details"]  # oru collection name

    # Insert a document
    data =test_text
    collection.insert_one(data)

    print("Document inserted successfully!")


A final function that calls all the functions we made earlier, making the process kind-of automated

In [10]:
def card_reader_final(image_location):
   # image_extract(image_location) might not need it to be called here
    text=classify_text(image_location)
    new_text=extract_json_content(text)
    add_db(new_text)


Testing with some Business Cards

In [ ]:
#an optical shop business card
#card_reader_final("business cards/bs_card_3.jpg")

In [12]:
#a random microsoft word business card
#card_reader_final("business cards/microsoft_business_card.png")

In [13]:
#testing with my uni ID card
#card_reader_final("business cards/cld_id_card.jpg")


In [14]:
#testing sugu business card
#card_reader_final("business cards/sugu_bs_card.jpg")

In [16]:
#testing pranav's uni id card
#card_reader_final("business cards/pranav_id_card.jpg")

Document inserted successfully!


UPDATE: The images need to be clear for it to be uploaded, I might consider going back and making the image grayscale to improve accuracy
- fancy texts are identified wrong sometimes

We have created a pipeline that takes in the image_location and then extracts the text and then converts into into json and stores it into a database

The outcomes will be provided in Readme.md file
